In [1]:
import pandas as pd
#import os
#pip install openpyxl
import numpy as np
#import scipy as sp


In [2]:
Data=pd.read_excel("DataCS-2018-2021.xlsx",
                   sheet_name=None,
                   usecols="B:P",
                   skipfooter=15,
                   header=7)



#LOADING SHEETS
Data18=Data['2018']
Data19=Data['2019']
Data20=Data['2020']
Data21=Data['2021']

#Data.keys()
#RENAME

# RENAME COLUMNS
new_columns = {0: 'Ma CK', 
               1: 'Ten Cong ty',
               2: 'San',
               3: 'Nhom nganh',
               4: 'Tong Tai san',
               5: 'Tong No phai tra',
               6: 'Von Chu so huu',
               7: 'No ngan han',
               8: 'No dai han',
               9: 'Tai san co dinh',
               10: 'Gia von hang ban',
               11: 'Loi nhuan truoc thue',
               12: 'Chi phi lai vay',
               13: 'Von hoa thi truong',
               14: 'ROA'               
              }

#RENAME and DROP ROWS

Data18=Data18.rename(columns={Data18.columns[i]: new_col for i, new_col in new_columns.items()})
Data19=Data19.rename(columns={Data19.columns[i]: new_col for i, new_col in new_columns.items()})
Data20=Data20.rename(columns={Data20.columns[i]: new_col for i, new_col in new_columns.items()})
Data21=Data21.rename(columns={Data21.columns[i]: new_col for i, new_col in new_columns.items()})

#DROP ROW 1-2
Data18.drop([0,1], axis=0, inplace=True)
Data19.drop([0,1], axis=0, inplace=True)
Data20.drop([0,1], axis=0, inplace=True)
Data21.drop([0,1], axis=0, inplace=True)

# LOC trong San Giao dich HOSE HNX UPCOM
San_GD = ['HOSE','HNX','UPCOM']
Data18 = Data18[Data18['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data19 = Data19[Data19['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data20 = Data20[Data20['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data21 = Data21[Data21['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)



In [3]:
# Tong hop Du lieu tu Nam 2018 - 2021
Data18['Nam']=2018
Data19['Nam']=2019
Data20['Nam']=2020
Data21['Nam']=2021
# Ket hop GDP va COVID
DataGDP_COVID=pd.read_csv("DataGDP_COVID.csv")
DataNganh=pd.read_csv("Index_NhomNganh.csv")
Data_Year=pd.read_csv("Data_ID_Year.csv")

# Data Tong hop
DataAll=pd.concat([Data18, Data19,Data20,Data21]).reset_index(drop=True)
DataAll=DataAll.merge(DataGDP_COVID, how='left', on='Nam')
DataAll=DataAll.merge(DataNganh, how='left', on='Nhom nganh')



In [4]:
#Don Vi la Unit: Million M
DataAll.iloc[:,4:13]=DataAll.iloc[:,4:13]//10**6 # Chia lay lam tron
#Von hoa thi truong, Unit: Ty dong, Billion  
DataAll['Von hoa thi truong']=DataAll['Von hoa thi truong'].apply(np.ceil)  #lam trong so tu nhien


In [5]:
# Tinh toan Nang cao
#GRWTH = MarketCap / Book value of Equity
DataAll['GRWTH']=DataAll['Von hoa thi truong']*1000/DataAll['Von Chu so huu']
# SIZE = natural logarithym LN(Total ASSETS)
DataAll['SIZE'] = np.log(DataAll['Tong Tai san'].astype('float'))
# EBIT = LNTT + Chi phi lai vay => PRO = EBIT/Total Assets ; RISK=Std(EBIT)
DataAll['EBIT']=DataAll['Loi nhuan truoc thue']+abs(DataAll['Chi phi lai vay'])
# Profitability PRO = EBIT/Tong Tai san
DataAll['PROF']=DataAll['EBIT']/DataAll['Tong Tai san']
# LIQD	Tổng tài sản ngắn hạn/ Nợ ngắn hạn
DataAll['LIQD']=DataAll['Tong Tai san']/DataAll['No ngan han']
# UNIQ	Giá vốn hàng bán/Tổng tài sản
DataAll['UNIQ']=abs(DataAll['Gia von hang ban'])/DataAll['Tong Tai san']
# Tính hữu hình	TANG	Tài sản cố định/ Tổng tài sản
DataAll['TANG']=DataAll['Tai san co dinh']/DataAll['Tong Tai san']
#BIEN PHU THUOC: STLEV, LTLEV, BLEV
DataAll['STLEV']=DataAll['No ngan han']/DataAll['Tong Tai san']
DataAll['LTLEV']=DataAll['No dai han']/DataAll['Tong Tai san']
DataAll['BLEV']=DataAll['Tong No phai tra']/DataAll['Tong Tai san']

In [16]:
#FILTER : Công ty có Total Asset >= 1000B and MarCap >= 1000B 
DataAll_2021=DataAll[DataAll['Nam'] == 2021].sort_values(by='Tong Tai san', ascending=False)
Top_Tickets=DataAll_2021.loc[(DataAll['Tong Tai san']>=1000*10**3) & (DataAll['Von hoa thi truong']>=1000)].reset_index(drop=True)
Top_Tickets=Top_Tickets.iloc[:,[0,1]]
Top_Tickets['ID_TICKET'] = np.arange(Top_Tickets.shape[0])
Top_Tickets=Top_Tickets.iloc[:,[2,0]]

DataAll_filter=pd.merge(DataAll, Top_Tickets, on='Ma CK', how='inner').reset_index(drop=True)
DataAll_filter


,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Tong No phai tra,Von Chu so huu,No ngan han,No dai han,Tai san co dinh,...,SIZE,EBIT,PROF,LIQD,UNIQ,TANG,STLEV,LTLEV,BLEV,ID_TICKET
0,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,53624854,22838548,30786305,7683984,15154564,18447261,...,17.797523,7713260,0.143837,6.978783,0.154697,0.344006,0.143291,0.282603,0.425895,8
1,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,58176195,21418960,36757234,6518193,14900767,16345334,...,17.878987,10252206,0.176227,8.925203,0.154615,0.280963,0.112042,0.256132,0.368174,8
2,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,56902290,19337072,37565218,3892509,15444563,14336420,...,17.856846,2096480,0.036844,14.618409,0.112088,0.251948,0.068407,0.271423,0.339829,8
3,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,54979483,17326792,37652691,3688003,13638788,12509813,...,17.822471,1079078,0.019627,14.907657,0.100717,0.227536,0.06708,0.248071,0.31515,8
4,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,64578613,30498935,34079678,15795515,14703420,29203764,...,17.983394,8911355,0.137992,4.088415,0.407352,0.45222,0.244594,0.227682,0.472276,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,DHT,Dược phẩm Hà Tây,HNX,Dịch vụ Tiêu dùng L1,1237590,484245,753345,479078,5166,149631,...,14.028676,103506,0.083635,2.583275,1.175948,0.120905,0.387106,0.004174,0.391281,214
916,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1343377,784614,558763,784614,0,356903,...,14.110697,300961,0.224033,1.71215,0.937955,0.265676,0.584061,0.0,0.584061,202
917,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1461188,794911,666276,794911,0,344912,...,14.194760,201081,0.137615,1.838178,0.754385,0.236049,0.544017,0.0,0.544017,202
918,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1659542,965082,694459,965082,0,322257,...,14.322052,75792,0.04567,1.719587,0.483084,0.194184,0.581535,0.0,0.581535,202


In [17]:
DataAll_filter=pd.merge(Data_Year, DataAll_filter, on='Nam', how='right').reset_index(drop=True)
DataAll_filter

,Nam,ID_Year,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Tong No phai tra,Von Chu so huu,No ngan han,...,SIZE,EBIT,PROF,LIQD,UNIQ,TANG,STLEV,LTLEV,BLEV,ID_TICKET
0,2018,2018,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,53624854,22838548,30786305,7683984,...,17.797523,7713260,0.143837,6.978783,0.154697,0.344006,0.143291,0.282603,0.425895,8
1,2019,2019,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,58176195,21418960,36757234,6518193,...,17.878987,10252206,0.176227,8.925203,0.154615,0.280963,0.112042,0.256132,0.368174,8
2,2020,2020,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,56902290,19337072,37565218,3892509,...,17.856846,2096480,0.036844,14.618409,0.112088,0.251948,0.068407,0.271423,0.339829,8
3,2021,2021,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,54979483,17326792,37652691,3688003,...,17.822471,1079078,0.019627,14.907657,0.100717,0.227536,0.06708,0.248071,0.31515,8
4,2018,2018,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,64578613,30498935,34079678,15795515,...,17.983394,8911355,0.137992,4.088415,0.407352,0.45222,0.244594,0.227682,0.472276,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,2021,2021,DHT,Dược phẩm Hà Tây,HNX,Dịch vụ Tiêu dùng L1,1237590,484245,753345,479078,...,14.028676,103506,0.083635,2.583275,1.175948,0.120905,0.387106,0.004174,0.391281,214
916,2018,2018,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1343377,784614,558763,784614,...,14.110697,300961,0.224033,1.71215,0.937955,0.265676,0.584061,0.0,0.584061,202
917,2019,2019,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1461188,794911,666276,794911,...,14.194760,201081,0.137615,1.838178,0.754385,0.236049,0.544017,0.0,0.544017,202
918,2020,2020,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1659542,965082,694459,965082,...,14.322052,75792,0.04567,1.719587,0.483084,0.194184,0.581535,0.0,0.581535,202


In [42]:
Output=DataAll_filter.iloc[:,[1,2,3,4,5,6,15,20,21,23,24,25,26,17,18,19,27,28,29]]
Output=Output.dropna().reset_index(drop=True)
Output

,ID_Year,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Von hoa thi truong,GRWTH,SIZE,PROF,LIQD,UNIQ,TANG,GDP,COVID,INDS,STLEV,LTLEV,BLEV
0,2018,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,53624854,186158,6.04678,17.797523,0.143837,6.978783,0.154697,0.344006,7.17,0,1,0.143291,0.282603,0.425895
1,2019,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,58176195,186158,5.064527,17.878987,0.176227,8.925203,0.154615,0.280963,7.02,0,1,0.112042,0.256132,0.368174
2,2020,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,56902290,186158,4.955595,17.856846,0.036844,14.618409,0.112088,0.251948,2.91,1,1,0.068407,0.271423,0.339829
3,2021,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,54979483,186158,4.944082,17.822471,0.019627,14.907657,0.100717,0.227536,2.58,1,1,0.06708,0.248071,0.31515
4,2018,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,64578613,152624,4.478446,17.983394,0.137992,4.088415,0.407352,0.45222,7.17,0,2,0.244594,0.227682,0.472276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,2021,DHT,Dược phẩm Hà Tây,HNX,Dịch vụ Tiêu dùng L1,1237590,1052,1.396439,14.028676,0.083635,2.583275,1.175948,0.120905,2.58,1,5,0.387106,0.004174,0.391281
909,2018,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1343377,1026,1.836199,14.110697,0.224033,1.71215,0.937955,0.265676,7.17,0,2,0.584061,0.0,0.584061
910,2019,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1461188,1026,1.539902,14.194760,0.137615,1.838178,0.754385,0.236049,7.02,0,2,0.544017,0.0,0.544017
911,2020,ACL,Thủy sản CL An Giang,HOSE,Hàng Tiêu dùng L1,1659542,1026,1.477409,14.322052,0.04567,1.719587,0.483084,0.194184,2.91,1,2,0.581535,0.0,0.581535


In [43]:
# EXPORT FILE DATA
with pd.ExcelWriter('Report.xlsx') as writer:
    DataAll.to_excel(writer, sheet_name='Origin')
    DataAll_filter.to_excel(writer, sheet_name='Filter')
    Output.to_excel(writer, sheet_name='Output')
    Top_Tickets.to_excel(writer,sheet_name='Top Tickets')